In [395]:
import duckdb
from soda_core.contracts import verify_contract_locally
from soda_duckdb import DuckDBDataSource

In [396]:
db_connection = duckdb.connect(database=':memory:')
cursor = db_connection.cursor()

In [397]:
cursor.execute("CREATE SCHEMA analytics")
cursor.execute("""
               CREATE TABLE analytics.fact_coffee_sales AS
               SELECT * FROM read_csv_auto('data/coffee_sales.csv')
""")


In [398]:
cursor.execute("""
    CREATE OR REPLACE TABLE analytics.fact_coffee_sales AS
    SELECT
        *,
        DAY(date) AS day,
        MONTH(date) AS month,
        YEAR(date) AS year
    FROM 
        read_csv_auto('data/coffee_sales.csv')
""")


In [399]:
# Inserting test rows
cursor.execute("""
    INSERT INTO analytics.fact_coffee_sales 
    (date, datetime, cash_type, card, money, coffee_name, day, month, year)
    VALUES 
    ('2025-03-24', '2025-03-24 09:00:00', 'PIX', NULL, 15.00, 'Espresso', 24, 3, 2025),
    ('2025-03-24', '2025-03-24 10:30:00', 'Crypto', 'BITCOIN-WALLET-XYZ', 50.00, 'Latte', 24, 3, 2025),
    ('2025-03-24', '2025-03-24 11:00:00', 'Voucher', 'TICKET-REFEICAO', 35.76, 'Cappuccino', 24, 3, 2025),
    ('2025-03-24', '2025-03-24 12:00:00', NULL, NULL, 10.00, 'Cortado', 24, 3, 2025)
""")


In [400]:
cursor.query("select median(money) from analytics.fact_coffee_sales")

┌───────────────┐
│ median(money) │
│    double     │
├───────────────┤
│         32.82 │
└───────────────┘

In [401]:
cursor.query("select avg(money) from analytics.fact_coffee_sales")

┌────────────────────┐
│     avg(money)     │
│       double       │
├────────────────────┤
│ 31.742401098900494 │
└────────────────────┘

In [402]:
cursor.query("select * from analytics.fact_coffee_sales")

┌────────────┬─────────────────────────┬───────────┬─────────────────────┬────────┬─────────────────────┬───────┬───────┬───────┐
│    date    │        datetime         │ cash_type │        card         │ money  │     coffee_name     │  day  │ month │ year  │
│    date    │        timestamp        │  varchar  │       varchar       │ double │       varchar       │ int64 │ int64 │ int64 │
├────────────┼─────────────────────────┼───────────┼─────────────────────┼────────┼─────────────────────┼───────┼───────┼───────┤
│ 2024-03-01 │ 2024-03-01 10:15:50.52  │ card      │ ANON-0000-0000-0001 │   38.7 │ Latte               │     1 │     3 │  2024 │
│ 2024-03-01 │ 2024-03-01 12:19:22.539 │ card      │ ANON-0000-0000-0002 │   38.7 │ Hot Chocolate       │     1 │     3 │  2024 │
│ 2024-03-01 │ 2024-03-01 12:20:18.089 │ card      │ ANON-0000-0000-0002 │   38.7 │ Hot Chocolate       │     1 │     3 │  2024 │
│ 2024-03-01 │ 2024-03-01 13:46:33.006 │ card      │ ANON-0000-0000-0003 │   28.9 │ Americ

In [403]:
result = verify_contract_locally(
    data_sources=[DuckDBDataSource.from_existing_cursor(cursor, name="duckdb")],
    contract_file_path="contracts/fact_coffee_sales.yml"
)

In [404]:
import pprint

print(f"--- VERIFICATION OF VALIDATION RESULT ---\n")

cv_result = result.contract_verification_results[0]

for i, res in enumerate(cv_result.check_results, 1):
    status = res.outcome
    nome = getattr(res.check, 'name', 'Check')
    
    print(f"Check #{i} | Status: {status} | Nome: {nome}")
    
    # If you want to see the complete check dictionary for debugging, uncomment the following line:
    #pprint.pprint(vars(res), indent=4)
    print("-" * 80)

--- VERIFICATION OF VALIDATION RESULT ---

Check #1 | Status: CheckOutcome.FAILED | Nome: No invalid values
--------------------------------------------------------------------------------
Check #2 | Status: CheckOutcome.FAILED | Nome: No missing values
--------------------------------------------------------------------------------
Check #3 | Status: CheckOutcome.FAILED | Nome: No invalid values
--------------------------------------------------------------------------------
Check #4 | Status: CheckOutcome.PASSED | Nome: Metric function meets threshold
--------------------------------------------------------------------------------
Check #5 | Status: CheckOutcome.FAILED | Nome: No invalid values
--------------------------------------------------------------------------------
Check #6 | Status: CheckOutcome.PASSED | Nome: Schema matches expected structure
--------------------------------------------------------------------------------
Check #7 | Status: CheckOutcome.FAILED | Nome: Row 

In [405]:
cv_result = result.contract_verification_results[0]
contract_obj = cv_result.contract

pprint.pprint(contract_obj.__dict__)

{'data_source_name': 'duckdb',
 'dataset_id': None,
 'dataset_name': 'fact_coffee_sales',
 'dataset_prefix': ['analytics'],
 'soda_qualified_dataset_name': 'duckdb/analytics/fact_coffee_sales',
 'source': YamlFileContentInfo(source_content_str='dataset: '
                                                  'duckdb/analytics/fact_coffee_sales\n'
                                                  '\n'
                                                  'columns:\n'
                                                  '  - name: datetime\n'
                                                  '    data_type: DATETIME\n'
                                                  '  - name: date\n'
                                                  '  - name: cash_type\n'
                                                  '    data_type: STRING\n'
                                                  '    checks:\n'
                                                  '      - invalid:\n'
                              